In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
DATA_ROOT = "/nemo/lab/znamenskiyp/home/shared/projects/"  # or None
# If in the lab, set DATA_ROOT to None, we'll find the data with flexiznam

# optional, can be None or the path to arial.ttf:
arial_font_path = None  # "/nemo/lab/znamenskiyp/home/shared/resources/fonts/arial.ttf"

In [ ]:
# Imports and setting matplotlib options
import numpy as np
from pathlib import Path
import tifffile as tf

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm

from brisc.manuscript_analysis import viral_library as virlib
from brisc.manuscript_analysis import start_density_sim as start_sim
from brisc.manuscript_analysis import rabies_cell_counting as rv_count
from brisc.manuscript_analysis import starter_cell_counting as sc_count
from brisc.manuscript_analysis import overview_image
from brisc.manuscript_analysis.utils import get_path, get_output_folder

# set matplotlib options
if arial_font_path is not None:
    arial_prop = fm.FontProperties(fname=arial_font_path)
    plt.rcParams["font.family"] = arial_prop.get_name()
    plt.rcParams.update({"mathtext.default": "regular"})  # make math mode also Arial
    fm.fontManager.addfont(arial_font_path)

matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs

In [ ]:
# Load viral library data
data_path = get_path(
    "barcode_diversity_analysis/collapsed_barcodes/", data_root=DATA_ROOT
)

libraries = {
    "Original plasmid library": virlib.load_library_data(data_path, "PBC20", 1, "bowtie"),
    "Original plasmid library resequenced": virlib.load_library_data(data_path, "Alex", 2, "bowtie"),
    "Amplified plasmid library": virlib.load_library_data(data_path, "Amplified", 2, "bowtie"),
}

#libraries_scale = {
#    # "2 plates #1": virlib.load_library_data(data_path, "RV31", 2, "bowtie"),
#    "2 plates #2": virlib.load_library_data(data_path, "RV32", 2, "bowtie"),
#    "12 plates": virlib.load_library_data(data_path, "RV35", 2, "bowtie"),
#}

In [ ]:
libraries

In [ ]:
# Plot Fig.1
fontsize_dict = {"title": 7, "label": 8, "tick": 6, "legend": 6}

line_width = 1.2
line_alpha = 1

cm = 1 / 2.54
fig = plt.figure(figsize=(17.4 * cm, 17.4 * cm), dpi=300)

save_path = get_output_folder(DATA_ROOT)
save_fig = False
figname = "fig1_plasmid_barcoding_schema_library"

# 1) Plot the plasmid and virus abundance histograms
ax_abundance = fig.add_axes([0.08, 0.8, 0.13, 0.13])
im = virlib.plot_barcode_counts_and_percentage(
    libraries,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_alpha=line_alpha,
    line_width=line_width,
    colors=[
        "dodgerblue",
        "darkorange",
        "green",
    ],
    ax=ax_abundance,
)
# Move the legend above the subplots to avoid occluding data
handles, labels = ax_abundance.get_legend_handles_labels()
existing_legend = ax_abundance.get_legend()
if existing_legend is not None:
    existing_legend.remove()
if handles:
    fig.legend(
        handles,
        labels,
        loc="upper center",
        bbox_to_anchor=(0.5, 1.02),
        ncol=len(handles),
        frameon=False,
        fontsize=fontsize_dict["legend"],
    )

# 2) Plot the plasmid and virus histograms
ax_unique = fig.add_axes([0.33, 0.8, 0.13, 0.13])
im = virlib.plot_unique_label_fraction(
    libraries,
    stride=50,
    max_cells=1e6,
    log_scale=True,
    min_max_percent_unique_range=(0.5, 1.0),
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_alpha=line_alpha,
    line_width=line_width,
    colors=[
        "dodgerblue",
        "darkorange",
        "green",
    ],
    ax=ax_unique,
    show_legend=False,
)
ax_unique.set_xticks([1, 1e3, 1e6])

if save_fig:
    save_path.mkdir(parents=True, exist_ok=True)
    fig.savefig(
        f"{save_path/figname}.pdf",
        format="pdf",
        dpi=600,
    )
    fig.savefig(
        f"{save_path/figname}.png",
        format="png",
    )
    print(f"Figure saved as {save_path/figname}")